In [ ]:
# Primeiro precisamos instalar o Selenium, uma ferramenta open-source para interagir de forma automatica com browsers web
!pip install -U selenium

In [ ]:
# Imports
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import numpy as np

In [ ]:
# Vamos utilizar o Google Chrome nesse scrapping, mas caso deseje utilizar qualquer outro navegador web, sugiro que pesquise na internet.

# Substitua o seu-usuario e sua-pasta-com-o-chrome-driver abaixo. O caminho precisa do arquivo precisa ser completo, não pode estar abreviado.
browser = webdriver.Chrome(executable_path=r"C:\Users\seu-usuario\sua-pasta-com-o-chrome-driver\chromedriver.exe")

In [ ]:
# Agora iremos criar a classe que será responsável pela extração dos dados
class ReclameAqui:

    # URL base do site do Reclame Aqui
    base_url = "https://www.reclameaqui.com.br/empresa/"
    
    # Inicializador da classe
    def __init__(self, driver, company):
        self.driver = driver
        self.company = company

    # Aqui iremos extrair as informações básicas das reclamações, como o título, o link da reclamação,
    # de que cidade ela foi feita, há quanto tempo foi feita e qual o seu status na plataforma.
    def extract_information(self, number_of_pages):
        url = self.base_url + self.company + "/lista-reclamacoes/?pagina="
        self.complaints, self.titles, self.links = [], [], []

        for i in range(1, number_of_pages + 1):
            self.driver.get(url + str(i))
            sleep(3)
            html = bs(self.driver.page_source, "html.parser")

            complaints_html = html.find_all("p", {"class": "text-detail"})
            page_complaints = [
                complaint.text.split("|") for complaint in complaints_html
            ]
            self.complaints.extend(page_complaints)

            titles_and_links = html.find_all(
                "a", {"class": "link-complain-id-complains"}
            )
            self.titles.extend([title.text.strip() for title in titles_and_links])
            self.links.extend([link.get("href") for link in titles_and_links])

    # Aqui iremos extrair o texto das reclamações dos clientes
    def extract_description(self):
        urls = [self.base_url + link[1:] for link in self.links]
        self.description = []
        for url in urls:
            self.driver.get(url)
            sleep(3)
            html = bs(self.driver.page_source, "html.parser")
            complaint = html.find("div", {"class": "complain-body"}).text.strip()
            self.description.append(complaint)

In [ ]:
# Inicializamos a classe sempre com o browser do ChromeDriver que criamos e com o nome da empresa que queremos extrair as informações do Reclame aqui

# - Exemplo: 
# Site do Reclame aqui: https://www.reclameaqui.com.br/empresa/nome-da-empresa/
# Como ficaria a inicialização da classe:
scrapping_reclame_aqui = ReclameAqui(browser, "nome-da-empresa")

In [ ]:
# Precisamos sempre extrair primeiro as informações porque elas nos darão o link a ser utilizado na função extrair_descricoes()

# Definimos quantas páginas queremos percorrer extraindo as informações da empresa
pages = 3
# Extraímos as informações
scrapping_reclame_aqui.extract_information(pages)

In [ ]:
# Agora podemos coletar os textos das reclamações dos clientes
scrapping_reclame_aqui.extract_description()

In [ ]:
# Armazenando os dados extraídos em arrays numpy
titles = np.array(scrapping_reclame_aqui.titles)
description = np.array(scrapping_reclame_aqui.description)
complaints = np.array(scrapping_reclame_aqui.complaints)
links = np.array(scrapping_reclame_aqui.links)